In [1]:
import sys
sys.path.append("/projects/robustmicrob/jlaw/projects/ESP/ESP_prediction_function/code")

In [2]:
from ES_prediction import *

FileNotFoundError: [Errno 2] No such file or directory: '../data/additional_data/all_substrates.pkl'

In [30]:
import pandas as pd
import numpy as np
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_rhea = pd.read_pickle("/projects/robustmicrob/jlaw/projects/ESP/ESP/data/reaction_data/RHEA_reaction_df.pkl")
df_rhea = df_rhea.dropna(subset=["EC_NUM"])
df_rhea

,RHEA ID,CHEBI IDs,CHEBI_ID_list,products,EC_NUM
0,10000.0,NaN,"[CHEBI:15377, CHEBI:16459]","[CHEBI:28938, CHEBI:31011]",3.5.1.50
4,10004.0,NaN,[CHEBI:17484],[CHEBI:16017],5.99.1.1
12,10012.0,NaN,"[CHEBI:58413, CHEBI:15377, CHEBI:15379]","[CHEBI:58682, CHEBI:16240]",1.5.3.6
16,10016.0,NaN,"[CHEBI:15377, CHEBI:16353]","[CHEBI:15354, CHEBI:30023, CHEBI:15378]",3.1.1.49
20,10020.0,NaN,"[CHEBI:15377, CHEBI:57951, CHEBI:58349]","[CHEBI:58321, CHEBI:15378, CHEBI:29985, CHEBI:...",1.5.1.10
...,...,...,...,...,...
53956,66372.0,NaN,"[CHEBI:15377, CHEBI:57597]","[CHEBI:17754, CHEBI:43474]",3.1.3.21
54120,66536.0,NaN,"[CHEBI:13193, CHEBI:157665, CHEBI:15377]","[CHEBI:157664, CHEBI:17499]",1.17.99.10
54124,66540.0,NaN,"[CHEBI:13193, CHEBI:17759, CHEBI:15377]","[CHEBI:17499, CHEBI:137057]",1.17.99.10
54164,66584.0,NaN,"[CHEBI:167166, CHEBI:13193, CHEBI:15377]","[CHEBI:167167, CHEBI:17499]",1.17.99.10


In [8]:
chebi_to_inchi_file = "/projects/robustmicrob/jlaw/projects/prot_stability_engineering/inputs/chemicals/chebiId_inchi.tsv"
chebi_to_inchi = pd.read_table(chebi_to_inchi_file)
chebi_to_inchi = dict(zip(chebi_to_inchi.CHEBI_ID, chebi_to_inchi.InChI))
print(len(chebi_to_inchi))

141645


In [18]:
names_file = "/projects/robustmicrob/jlaw/projects/prot_stability_engineering/inputs/chemicals/names.tsv.gz"
df_names = pd.read_table(names_file)
df_names = df_names[df_names.LANGUAGE == 'en']
df_names = df_names[df_names.SOURCE == 'UniProt']
df_names.head(2)

,ID,COMPOUND_ID,TYPE,SOURCE,NAME,ADAPTED,LANGUAGE
11886,16328,15382,NAME,UniProt,(4R)-limonene,F,en
11887,16329,15385,NAME,UniProt,"(1S,8aR)-delta-cadinene",F,en


In [19]:
df_names[df_names.COMPOUND_ID == 15377]

,ID,COMPOUND_ID,TYPE,SOURCE,NAME,ADAPTED,LANGUAGE
16764,37353,15377,NAME,UniProt,H2O,F,en


In [23]:
substrates = [c for chebi_id_list in df_rhea.CHEBI_ID_list.values for c in chebi_id_list]
S = pd.Series(substrates)
S.value_counts().head(40)

CHEBI:15377     1524
CHEBI:15379     1069
CHEBI:15378      916
CHEBI:30616      621
CHEBI:57540      507
CHEBI:59789      468
CHEBI:58349      426
CHEBI:57618      236
CHEBI:57783      188
CHEBI:16810      177
CHEBI:57288      148
CHEBI:57945      135
CHEBI:58885      135
CHEBI:175763     123
CHEBI:13193      115
CHEBI:33738       92
CHEBI:57287       90
CHEBI:43474       80
CHEBI:29033       67
CHEBI:17499       66
CHEBI:57384       56
CHEBI:57705       54
CHEBI:57623       47
CHEBI:29985       47
CHEBI:15361       47
CHEBI:29034       44
CHEBI:58057       43
CHEBI:58339       40
CHEBI:37565       40
CHEBI:66914       37
CHEBI:33737       36
CHEBI:58756       36
CHEBI:29969       34
CHEBI:16240       32
CHEBI:58359       31
CHEBI:37563       31
CHEBI:58342       30
CHEBI:64479       29
CHEBI:57692       29
CHEBI:57925       29
dtype: int64

In [ ]:
# try to get a list of cofactors in reactions
cofactor_chebi_ids = {
    # inorganic
    29034: 'Fe(3+)',
     29033: 'Fe(2+)',
    # vitamins and derivatives
     57287: 'CoA',
     57288: 'acetyl-CoA',
     58342: 'an acyl-CoA',
     57692: 'FAD',
     58307: 'FADH2',
     57618: 'FMNH2', # reduced flavin mononucleotide
     57540: 'NAD(+)',
    57945: 'NADH',
     58349: 'NADP(+)',
     57783: 'NADPH',
     58057: '(2E)-geranyl diphosphate',
     58885: 'UDP-alpha-D-glucose',
     57705: 'UDP-N-acetyl-alpha-D-glucosamine',
     58339: "3'-phosphoadenylyl sulfate",  # (PAPS) is a derivative of adenosine monophosphate (AMP). It is the most common coenzyme in sulfotransferase reactions

    # others
     15379: 'O2',
     15377: 'H2O',
     16240: 'H2O2',
     15378: 'H(+)',
    28938: 'NH4(+)',
     30616: 'ATP', 
     456216: 'ADP',
     37565: 'GTP',
     37563: 'CTP',  # Cytidine triphosphate (CTP) is a coenzyme in metabolic reactions such as: Glycerophospholipid synthesis, Protein glycosylation, RNA formation
     57384: 'malonyl-CoA',
     57623: 'dimethylallyl diphosphate',
    58756: '(2E,6E,10E)-geranylgeranyl diphosphate',
     43474: 'phosphate',
    128769: 'isopentenyl diphosphate',
    33019: 'diphosphate',
    59789: 'S-adenosyl-L-methionine',  # (SAMe) is a common cosubstrate involved in methyl group transfers
}
# not cofactors
# 15361: 'pyruvate',
# 57925: 'glutathione',
# 16810: '2-oxoglutarate',
# amino acids
# 29985: 'L-glutamate',
# 58359: 'L-glutamine',

In [57]:
# common_ids = S.value_counts().head(80).tail(40).index
# common_ids = [int(c.split(':')[1]) for c in common_ids]
# common_mols2 = df_names[df_names.COMPOUND_ID.isin(common_ids)]
# dict(zip(common_mols2.COMPOUND_ID, common_mols2.NAME))

In [58]:
common_ids = S.value_counts().head(40).index
common_ids = [int(c.split(':')[1]) for c in common_ids]
common_mols = df_names[df_names.COMPOUND_ID.isin(common_ids)]
common_chebis = ['CHEBI:'+str(c_id) for c_id in common_mols.COMPOUND_ID]
common_mols.head()

,ID,COMPOUND_ID,TYPE,SOURCE,NAME,ADAPTED,LANGUAGE
12220,18772,13193,NAME,UniProt,A,F,en
12230,18812,17499,NAME,UniProt,AH2,F,en
12240,18899,29034,NAME,UniProt,Fe(3+),F,en
12241,18900,29033,NAME,UniProt,Fe(2+),F,en
16760,37349,15379,NAME,UniProt,O2,F,en


In [ ]:
# see if 

In [28]:
import os
os.chdir("../../")
os.getcwd()

'/lustre/eaglefs/projects/robustmicrob/jlaw/projects/prot_stability_engineering'

In [31]:
inputs_dir = "inputs/brenda"
# This file has the ph range data + optimum ph data
# data_file = Path(inputs_dir, "20230726_ph_activity_data_merged.csv")
# data
data_file = Path(inputs_dir, "ph_range/20230719_ph_range_processed.csv")
print(data_file)
data = pd.read_csv(data_file, index_col=0)
print(len(data))

inputs/brenda/ph_range/20230719_ph_range_processed.csv
5842


In [32]:
data.head(2)

,ec_num,uniprot_id,species,ph_min,ph_max,references,comments
0,1.1.1.1,B2KJ46,Candida parapsilosis,4.0,7.5,17435004,"pH 4.0: about 80% of maximal activity, pH 7.5..."
1,1.1.1.1,Q6L0S1,Picrophilus torridus,7.0,8.0,17989975,"pH 7: 41% of maximal activity, pH 8: 59% of m..."


In [33]:
data.ec_num.value_counts()

3.2.1.8     75
3.2.1.4     59
3.2.1.1     48
3.2.1.14    43
3.2.1.21    36
            ..
1.2.1.46     1
1.2.1.47     1
1.2.1.77     1
1.2.1.8      1
3.5.4.10     1
Name: ec_num, Length: 1766, dtype: int64

In [18]:
names_file = "/projects/robustmicrob/jlaw/projects/prot_stability_engineering/inputs/chemicals/names.tsv.gz"
df_names = pd.read_table(names_file)
df_names = df_names[df_names.LANGUAGE == 'en']
df_names = df_names[df_names.SOURCE == 'UniProt']
df_names.head(2)

,ID,COMPOUND_ID,TYPE,SOURCE,NAME,ADAPTED,LANGUAGE
11886,16328,15382,NAME,UniProt,(4R)-limonene,F,en
11887,16329,15385,NAME,UniProt,"(1S,8aR)-delta-cadinene",F,en


In [45]:
# load the ec number to rhea reaction mapping
ec_to_rhea_file = "/projects/robustmicrob/jlaw/projects/prot_stability_engineering/inputs/chemicals/ec_to_rhea_mapping.tsv"
ec_to_rhea = pd.read_table(ec_to_rhea_file)
ec_to_rhea

,ec_num,rhea_id,ec_class,equation,enzymes
0,5.3.3.8,RHEA:23716,EC:5.3.3.8 Delta(3)-Delta(2)-enoyl-CoA isomerase,(3Z)-dodecenoyl-CoA = (2E)-dodecenoyl-CoA,0
1,6.4.1.3,RHEA:23720,EC:6.4.1.3 propionyl-CoA carboxylase,ATP + hydrogencarbonate + propanoyl-CoA = (S)-...,7149
2,1.14.99.1,RHEA:23728,EC:1.14.99.1 prostaglandin-endoperoxide synthase,"(5Z,8Z,11Z,14Z)-eicosatetraenoate + AH2 + 2 O2...",17
3,4.1.1.33,RHEA:23732,EC:4.1.1.33 diphosphomevalonate decarboxylase,(R)-5-diphosphomevalonate + ATP = ADP + CO2 + ...,2972
4,1.1.3.37,RHEA:23756,"EC:1.1.3.37 D-arabinono-1,4-lactone oxidase","D-arabinono-1,4-lactone + O2 = dehydro-D-arabi...",981
...,...,...,...,...,...
1830,5.5.1.2,RHEA:23656,"EC:5.5.1.2 3-carboxy-cis,cis-muconate cycloiso...","2-(carboxymethyl)-5-oxo-2,5-dihydro-2-furoate ...",4
1831,3.6.1.15,RHEA:23680,EC:3.6.1.15 nucleoside-triphosphate phosphatase,a ribonucleoside 5'-triphosphate + H2O = a rib...,51771
1832,3.5.4.2,RHEA:23688,EC:3.5.4.2 adenine deaminase,adenine + H(+) + H2O = hypoxanthine + NH4(+),24221
1833,1.13.99.1,RHEA:23696,EC:1.13.99.1 inositol oxygenase,myo-inositol + O2 = D-glucuronate + H(+) + H2O,3812


In [46]:
ec_to_rhea['ec_num'].value_counts()

2.7.4.6     12
2.4.2.1      9
3.6.1.9      9
3.1.3.25     7
3.6.1.6      6
            ..
2.6.1.59     1
3.1.1.35     1
3.5.2.9      1
4.2.1.3      1
3.5.4.9      1
Name: ec_num, Length: 1440, dtype: int64

In [ ]:
data['rhea_ids'] = data.ec_num.apply(lambda ec_num: ec_to_rhea[ec_to_rhea.ec_num == ec_num]['rhea_id'].values)
# data['substrates'] = data.ec_num.apply(lambda ec_num: df_rhea[df_rhea.EC_NUM == ec_num].CHEBI_ID_list.values)
data.head(2)

,ec_num,uniprot_id,species,ph_min,ph_max,references,comments,substrates,rhea_reactions,rhea_ids
0,1.1.1.1,B2KJ46,Candida parapsilosis,4.0,7.5,17435004,"pH 4.0: about 80% of maximal activity, pH 7.5...","[[CHEBI:35681, CHEBI:57540], [CHEBI:16236, CHE...","[10740.0, 25290.0]","[RHEA:25290, RHEA:10740]"
1,1.1.1.1,Q6L0S1,Picrophilus torridus,7.0,8.0,17989975,"pH 7: 41% of maximal activity, pH 8: 59% of m...","[[CHEBI:35681, CHEBI:57540], [CHEBI:16236, CHE...","[10740.0, 25290.0]","[RHEA:25290, RHEA:10740]"


In [53]:
data[data.rhea_reactions.apply(len) == 0].ec_num.value_counts()

3.2.1.8      75
3.2.1.4      59
3.2.1.1      48
3.2.1.14     43
3.1.1.3      36
             ..
3.2.1.171     1
3.1.2.20      1
3.1.21.10     1
3.1.3.23      1
1.1.1.116     1
Name: ec_num, Length: 426, dtype: int64

In [41]:
data[data.rhea_reactions.apply(len) > 10]

,ec_num,uniprot_id,species,ph_min,ph_max,references,comments,substrates,rhea_reactions
1191,2.7.4.6,Q81SV8,Bacillus anthracis,3.5,10.5,19241473,None,"[[CHEBI:57930, CHEBI:30616], [CHEBI:30616, CHE...","[18113.0, 25098.0, 25237.0, 27674.0, 27678.0, ..."


In [52]:
# data['num_rxns'] = 
data.rhea_reactions.apply(len).value_counts()
data.rhea_ids.apply(len).value_counts()

1     3280
0     1784
2      549
3      137
4       50
6       21
5        9
9        7
7        4
12       1
Name: rhea_ids, dtype: int64